In [1]:
from desdeo_emo.EAs.RVEA import RVEA
from desdeo_problem.Variable import variable_builder
from desdeo_problem.Objective import _ScalarObjective, VectorObjective
from desdeo_problem.Problem import MOProblem

import numpy as np
import pandas as pd
from desdeo_emo.othertools.plotlyanimate import animate_init_, animate_next_

In [2]:
# create the problem
def f_1(x):
    return 4.07 + 2.27 * x[:, 0]

def f_2(x):
    return 2.60 + 0.03*x[:, 0] + 0.02*x[:, 1] + 0.01 / (1.39 - x[:, 0]**2) + 0.30 / (1.39 - x[:, 1]**2)

def f_3(x):
    return 8.21 - 0.71 / (1.09 - x[:, 0]**2)

def f_4(x):
    return 0.96 - 0.96 / (1.09 - x[:, 1]**2)

# def f_5(x):
    # return -0.96 + 0.96 / (1.09 - x[:, 1]**2)

def f_5(x):
    return np.max([np.abs(x[:, 0] - 0.65), np.abs(x[:, 1] - 0.65)], axis=0)

In [3]:
f1 = _ScalarObjective(name="f1", evaluator=f_1, maximize=True)
f2 = _ScalarObjective(name="f2", evaluator=f_2, maximize=True)
f3 = _ScalarObjective(name="f3", evaluator=f_3, maximize=True)
f4 = _ScalarObjective(name="f4", evaluator=f_4, maximize=True)
f5 = _ScalarObjective(name="f5", evaluator=f_5, maximize=False)

In [4]:
varsl = variable_builder(["x_1", "x_2"],
        initial_values=[0.5, 0.5],
        lower_bounds=[0.3, 0.3],
        upper_bounds=[1.0, 1.0]
        )

In [5]:
problem = MOProblem(variables=varsl, objectives=[f1, f2, f3, f4, f5])

In [6]:
evolver = RVEA(problem, interact=True, n_iterations=5, n_gen_per_iter=100)
figure = animate_init_(evolver.population.objectives, filename="river.html")

Plot saved as:  river.html
View the plot by opening the file in browser.
To view the plot in Jupyter Notebook, use the IFrame command.


In [7]:
plot, pref = evolver.requests()

In [8]:
print(plot.content["dimensions_data"])

             f1       f2       f3          f4         f5
minimize     -1       -1       -1          -1          1
ideal     6.337  3.40722  7.49915 -0.00143157  0.0235701
nadir      None     None     None        None       None


In [9]:
print(pref.content['message'])

Provide a reference point worse than or equal to the ideal point:
f1         6.337
f2       3.40722
f3       7.49915
f4   -0.00143157
f5     0.0235701
Name: ideal, dtype: object
The reference point will be used to focus the reference vectors towards the preferred region.
If a reference point is not provided, the previous state of the reference vectors is used.
If the reference point is the same as the ideal point, the ideal point, the reference vectors are spread uniformly in the objective space.


In [10]:
pref.response = pd.DataFrame([[6.3,3.3,7,-2,0.3]],
                             columns=pref.content['dimensions_data'].columns)

In [11]:
plot, pref = evolver.iterate(pref)
figure = animate_next_(
    plot.content['data'].values,
    figure,
    filename="river.html",
    generation=evolver._iteration_counter,
)

message = (f"Current generation number:{evolver._current_gen_count}. "
           f"Is looping back recommended: {'Yes' if evolver.continue_evolution() else 'No'}")
print(message)

Current generation number:100. Is looping back recommended: Yes
